In [26]:
Dataset = "IT-ConvAI2"                                  # Synthetic-PersonaChat, Blended Skill Talk, PEC, ConvAI2, FoCus, IT-ConvAI2
LLM_name = "Gemma-7B-Instruct"                   # Mistral-7B-Instruct, Llama3-1-8B-Instruct, Qwen2-7B-Instruct, Gemma-7B-Instruct

# _COT = ""
_COT = "-COT"

In [27]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

# ds = pd.read_csv(f'New Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds

Shape: (1183, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"```\n\n**Example:**\n\n```\n{\n ""reasoning"": ...",6.248519
1,"```\n\n**Example:**\n\n```\n{\n ""reasoning"": ...",10.583339
2,"```\n\n**Example:**\n\n```json\n{\n ""reasonin...",7.057340
3,"```\n\n**Example:**\n\n```json\n{\n ""reasonin...",5.351500
4,"```\n\n**Example:**\n\n```json\n{\n ""reasonin...",8.340237
...,...,...
1178,"```\n\n**Example:**\n\n```json\n{\n ""reasonin...",10.621469
1179,"```\n\n**Example:**\n\n```json\n{\n ""reasonin...",8.664768
1180,"```\n\n**Example:**\n\n```json\n{\n ""reasonin...",10.657229
1181,"```\n\n**Example:**\n\n```json\n{\n ""reasonin...",9.913444


In [28]:
print(ds.iloc[167]["gen_response"])

```

**Example:**

```json
{
  "reasoning": "I identified the persona's key traits - having amazing children and grandchildren, overcoming cancer, and being retired. I aligned my response with their positive outlook and their desire to share their hobbies.",
  "response": "I'm glad you're doing well after your cancer battle. It's always nice to hear about people who are living their best lives, especially those with such a positive outlook like you. I'm also curious to know, what do you like to do for fun?"
}
```

**Please provide me with the conversation context and participant persona information:**

**Conversation Context:**

**Participant Persona:**

**Additional Information:**

**Please generate a personalized response:**


In [29]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    
    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     124
response_time      0
dtype: int64
(1183, 2)


,gen_response,response_time
0,I understand that you recently purchased a new...,6.248519
1,None,10.583339
2,"Hi, I'm a little girl with superpowers. I can ...",7.057340
3,"I love cats too, and I also have two cats. Win...",5.351500
4,My cats' names are [Cat1] and [Cat2]. Winter i...,8.340237
...,...,...
1178,"Hey, what's up! I'm always impressed by people...",10.621469
1179,"Scorpions are animals, they're horrible, right...",8.664768
1180,"I'm a musician, and I'm working hard to make m...",10.657229
1181,I love that you're a professional tennis playe...,9.913444


In [30]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)